In [1]:
from data import DataLoader
from model import model_fn
from config import args

import tensorflow as tf
import numpy as np
import json
import pprint


def main():
    tf.logging.set_verbosity(tf.logging.INFO)
    print(json.dumps(args, indent=4))

    train_dl = DataLoader(
        path='./temp/qa5_three-arg-relations_train.txt',
        is_training=True)
    test_dl = DataLoader(
        path='./temp/qa5_three-arg-relations_test.txt',
        is_training=False, vocab=train_dl.vocab, params=train_dl.params)

    model = tf.estimator.Estimator(model_fn, params=train_dl.params)
    model.train(train_dl.input_fn())
    gen = model.predict(test_dl.input_fn())
    preds = np.concatenate(list(gen))
    preds = np.reshape(preds, [test_dl.data['size'], 2])
    print('Testing Accuracy:', (test_dl.data['val']['answers'][:, 0] == preds[:, 0]).mean())
    demo(test_dl.demo, test_dl.vocab['idx2word'], preds)


def demo(demo, idx2word, ids, demo_idx=3):
    demo_i, demo_q, demo_a = demo
    print()
    pprint.pprint(demo_i[demo_idx])
    print()
    print('Question:', demo_q[demo_idx])
    print()
    print('Ground Truth:', demo_a[demo_idx])
    print()
    print('- '*12)
    print('Machine Answer:', [idx2word[id] for id in ids[demo_idx]])


if __name__ == '__main__':
    main()

{
    "n_epochs": 10,
    "batch_size": 64,
    "embed_dim": 80,
    "hidden_size": 80,
    "dropout_rate": 0.1,
    "n_hops": 2,
    "clip_norm": 5.0
}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpp1ytku6h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11c606160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
==> Memory Episode 0
==> Memory Episode 1
==> Memory Episode 0
==> Memory Episode 1
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow: